## Complete Numeric one 15-state ESKF step (propagate + camera update)

Below is **one complete 15-state ESKF step** (propagate + camera update) with **numbers everywhere**, including **gyro and accel bias updates**.

We assume **monocular VO** gives a relative pose $(R_{vo},t_{vo})$ between two camera frames, and you **force scale** $|t_{vo}|=1$ (so it's a *pseudo-metric* translation). We turn that into a pseudo absolute pose measurement at time $k{+}1$ by composing with the known pose at time $k$. To keep the example minimal, we take the pose at $k$ as identity in the world.

---

# 1) State and ordering

### Nominal (stored)

$$
\hat x = (\hat p,\ \hat v,\ \hat R,\ \hat b_g,\ \hat b_a)
$$

### Error-state (for covariance only)

$$
\delta x = (\delta p,\ \delta v,\ \delta\theta,\ \delta b_g,\ \delta b_a)\in\mathbb{R}^{15}
$$

Ordering used below:
$$
[\delta p(0:3),\ \delta v(3:6),\ \delta\theta(6:9),\ \delta b_g(9:12),\ \delta b_a(12:15)]
$$

---

# 2) Given values (time k → k+1)

$$
\Delta t = 0.1\ \text{s},\quad g=\begin{bmatrix}0\\0\\-9.81\end{bmatrix}
$$

Nominal at $k$:
$$
\hat p=\begin{bmatrix}0\\0\\0\end{bmatrix},\ \hat v=\begin{bmatrix}1\\0\\0\end{bmatrix},\ \hat R=I
$$

$$
\hat b_g=\begin{bmatrix}0.02\\-0.01\\0.005\end{bmatrix}\ \text{rad/s},\quad \hat b_a=\begin{bmatrix}0.1\\-0.05\\0.02\end{bmatrix}\ \text{m/s}^2
$$

IMU readings:
$$
\omega_m=\begin{bmatrix}0\\0\\1\end{bmatrix}\ \text{rad/s},\quad a_m=\begin{bmatrix}0.4\\0\\9.81\end{bmatrix}\ \text{m/s}^2
$$

Bias-corrected:
$$
\omega=\omega_m-\hat b_g=\begin{bmatrix}-0.02\\0.01\\0.995\end{bmatrix},\quad f=a_m-\hat b_a=\begin{bmatrix}0.3\\0.05\\9.79\end{bmatrix}
$$

---

# 3) Nominal propagation (what you code)

### Rotation

$$
\phi=\omega\Delta t=\begin{bmatrix}-0.002\\0.001\\0.0995\end{bmatrix}
$$

$$
\hat R^+ = \hat R\Exp(\phi^\wedge)=\Exp(\phi^\wedge) \approx \begin{bmatrix}0.995053 & -0.099337 & 0.000899\\0.099335 & 0.995052 & 0.002046\\-0.001098 & -0.001947 & 0.999998\end{bmatrix}
$$

### Acceleration in world, velocity, position

$$
a = \hat R^+ f + g \approx \begin{bmatrix}0.302350\\0.099587\\-0.020451\end{bmatrix}
$$

$$
\hat v^+ = \hat v + a\Delta t \approx \begin{bmatrix}1.030235\\0.009959\\-0.002045\end{bmatrix}
$$

$$
\hat p^+ = \hat p + \hat v\Delta t + \tfrac12 a\Delta t^2 \approx \begin{bmatrix}0.101512\\0.000498\\-0.000102\end{bmatrix}
$$

Biases (nominal) are usually random-walk, so:
$$
\hat b_g^+=\hat b_g,\quad \hat b_a^+=\hat b_a
$$

---

# 4) Camera (monocular) gives relative pose between frames

Given VO result:

* relative rotation: yaw $0.09$ rad
  $$
  R_{vo}=\Exp([0,0,0.09]^\wedge)
  $$
* relative translation direction scaled to 1:
  $$
  t_{vo}= \begin{bmatrix}1\\0\\0\end{bmatrix}
  $$

Assume the world pose at time $k$ is identity (for this example), so the pseudo absolute measurement at $k{+}1$ is:
$$
R_{meas}=R_{vo},\quad p_{meas}=t_{vo}
$$

---

# 5) Measurement residual used in ESKF (6D)

We measure position and orientation:
$$
z = \begin{bmatrix} p_{meas}\\ R_{meas} \end{bmatrix},\quad h(\hat x^+) = \begin{bmatrix} \hat p^+\\ \hat R^+ \end{bmatrix}
$$

Residual:
$$
r_p = p_{meas}-\hat p^+ = \begin{bmatrix}0.898488\\-0.000498\\0.000102\end{bmatrix}
$$

$$
r_\theta = \Log\big(R_{meas}(\hat R^+)^T\big) = \begin{bmatrix}0.002042\\-0.000909\\-0.009500\end{bmatrix}\ \text{rad}
$$

Stack:
$$
r=\begin{bmatrix} r_p\\ r_\theta \end{bmatrix}\in\mathbb{R}^6
$$

---

# 6) Linearized measurement Jacobian H

Because $p$ depends on $\delta p$ and orientation depends on $\delta\theta$:
$$
H=\begin{bmatrix} I_3 & 0 & 0 & 0 & 0\\ 0 & 0 & I_3 & 0 & 0 \end{bmatrix} \in\mathbb{R}^{6\times 15}
$$

---

# 7) Linearized error dynamics Jacobian F

Discrete-time (Euler) linearized error model:
$$
\delta p^+=\delta p+\delta v\Delta t
$$

$$
\delta v^+=\delta v-\hat R^+[f]_\times\delta\theta\Delta t-\hat R^+\delta b_a\Delta t
$$

$$
\delta\theta^+=(I-[\omega]_\times\Delta t)\delta\theta-\delta b_g\Delta t
$$

$$
\delta b_g^+=\delta b_g,\quad \delta b_a^+=\delta b_a
$$

So $F$ blocks:

* $F_{pv}=I\Delta t$
* $F_{v\theta}=-\hat R^+[f]_\times\Delta t$
* $F_{v b_a}=-\hat R^+\Delta t$
* $F_{\theta\theta}=I-[\omega]_\times\Delta t$
* $F_{\theta b_g}=-I\Delta t$


---

# 8) Covariance prediction $(P^-=FPF^T+Q)$

Initial covariance (example, with some cross-correlation so biases can update):

* $P_{pp}=0.01I$
* $P_{vv}=0.04I$
* $P_{\theta\theta}=0.0025I$
* $P_{b_gb_g}=10^{-4}I$
* $P_{b_ab_a}=10^{-2}I$
* $P_{\theta b_g}=5\cdot10^{-5}I$
* $P_{v b_a}=2\cdot10^{-3}I$

Noise (simple diagonal discretization):

* gyro white: $\sigma_g=0.01$
* accel white: $\sigma_a=0.1$
* gyro RW: $\sigma_{bg}=0.001$
* accel RW: $\sigma_{ba}=0.01$

So:

* $Q_{\theta\theta}=\sigma_g^2\Delta t,I=10^{-5}I$
* $Q_{vv}=\sigma_a^2\Delta t,I=0.001I$
* $Q_{b_gb_g}=\sigma_{bg}^2\Delta t,I=10^{-7}I$
* $Q_{b_ab_a}=\sigma_{ba}^2\Delta t,I=10^{-5}I$

After prediction, diagonals become (rounded):
$$
\mathrm{diag}(P^-)\approx \begin{aligned} P_{pp}&:\ [0.0104,0.0104,0.0104]\\ P_{vv}&:\ [0.04310,0.04310,0.04070]\\ P_{\theta\theta}&:\ [0.002526,0.002526,0.002501]\\ P_{b_gb_g}&:\ [0.0001001,\dots]\\ P_{b_ab_a}&:\ [0.01001,\dots] \end{aligned}
$$

---

# 9) Measurement noise for monocular pose

Because translation scale is forced to 1, we treat it as noisy:
$$
R_m=\mathrm{diag}(0.25,0.25,0.25,\ 0.01,0.01,0.01)
$$

---

# 10) Kalman update (this produces bias corrections)

$$
S = HP^-H^T + R_m
$$

$$
K = P^-H^T S^{-1}
$$

$$
\boxed{\delta\hat x = Kr}
$$

Numerical result (the actual correction vector you compute):

### Position / velocity / orientation correction

$$
\delta\hat p=\begin{bmatrix}0.035884\\-0.0000199\\0.00000408\end{bmatrix}
$$

$$
\delta\hat v=\begin{bmatrix}0.013649\\-0.000496\\0.00000914\end{bmatrix}
$$

$$
\delta\hat\theta=\begin{bmatrix}0.000411\\-0.000183\\-0.001901\end{bmatrix}\ \text{rad}
$$

### Bias updates (what you asked for)

These come from the cross-covariances inside $P^-$:

$$
\boxed{\delta\hat b_g=\begin{bmatrix}+6.85\cdot10^{-6}\\-2.17\cdot10^{-6}\\-3.04\cdot10^{-5}\end{bmatrix}\ \text{rad/s}}
$$

$$
\boxed{\delta\hat b_a=\begin{bmatrix}+6.90\cdot10^{-4}\\-3.82\cdot10^{-7}\\+7.85\cdot10^{-8}\end{bmatrix}\ \text{m/s}^2}
$$

(They are small in a single step; over many frames they accumulate.)

---

# 11) Injection into nominal (what you code)

$$
\hat p \leftarrow \hat p^+ + \delta\hat p \Rightarrow \hat p = \begin{bmatrix}0.137396\\0.000478\\-0.0000982\end{bmatrix}
$$

$$
\hat v \leftarrow \hat v^+ + \delta\hat v \Rightarrow \hat v = \begin{bmatrix}1.043884\\0.009463\\-0.002036\end{bmatrix}
$$

$$
\hat R \leftarrow \hat R^+ \Exp((\delta\hat\theta)^\wedge) \quad\text{(stays on }SO(3)\text{)}
$$

$$
\hat b_g \leftarrow \hat b_g + \delta\hat b_g = \begin{bmatrix}0.02000685\\-0.01000217\\0.00496960\end{bmatrix}
$$

$$
\hat b_a \leftarrow \hat b_a + \delta\hat b_a = \begin{bmatrix}0.10069008\\-0.05000038\\0.02000008\end{bmatrix}
$$

Covariance update (Joseph form, robust):
$$
P \leftarrow (I-KH)P^-(I-KH)^T + K R_m K^T
$$

Reset is only conceptual: the **mean** of the new error is defined to be zero.

---

## Important note about monocular "T scaled to 1"

With a single camera, the translation from VO is only a **direction** (scale is unobservable without extra info). Setting $|t|=1$ makes it a **pseudo measurement**, so you must reflect that by using a **large position noise** (like the $0.25$ used above). Otherwise the filter will over-trust fake metric scale.

If you want, I can redo the same numeric example using a more realistic monocular measurement: **rotation + translation direction only** (bearing constraint), which avoids pretending scale is known.